# CSE416 - Final Project: Link Prediction

name: Angelica Tao Zhu, Yue Lin

In [4]:
import networkx as nx
import numpy as np

# load General Relativity and Quantum Cosmology collaboration network from text file
collaboration_network = nx.Graph()
with open("dataset/ca-GrQc.txt", "r") as fp:
    line = fp.readline()
    # skip comments
    while line[0] == '#':
        line = fp.readline()
        
    while line:
        edge = line.split()
        collaboration_network.add_edge(edge[0], edge[1])
        line = fp.readline()

# load facebook ego network
facebook_ego = nx.Graph()
with open("dataset/facebook_combined.txt", "r") as fp:
    line = fp.readline()
        
    while line:
        edge = line.split()
        facebook_ego.add_edge(edge[0], edge[1])
        line = fp.readline()
        
# load email-Eu-core temporal network
email_eu = nx.Graph()
with open("dataset/email-Eu-core-temporal.txt", "r") as fp:
    line = fp.readline()
    while line:
        edge = line.split()
        email_eu.add_edge(edge[0], edge[1])
        line = fp.readline()

In [5]:
def loadGraph(fileName):
    g = nx.Graph()
    with open("dataset/" + fileName + ".txt", "r") as fp:
        line = fp.readline()
        
        # skip comments
        while line[0] == '#':
            line = fp.readline()
            
        while line:
            edge = line.split()
            g.add_edge(edge[0], edge[1])
            line = fp.readline()
    return g

In [6]:
def splitFile(fileName, percentage):
    filePath = 'dataset/' + fileName + '.txt'
    num_lines = sum(1 for line in open(filePath))
    num_train_lines = int(num_lines * percentage)
    with open(filePath) as fp:
        count = 1
        line = fp.readline()
        
        trainf = open('dataset/' + fileName + '_train.txt', "a")
        while count < num_train_lines:
            trainf.write(line)
            line = fp.readline()
            count += 1
        
        testf = open('dataset/' + fileName + '_test.txt', "a")
        while line:
            testf.write(line)
            line = fp.readline()

In [7]:
def _apply_prediction(G, func, ebunch=None):
    """Applies the given function to each edge in the specified iterable
    of edges.

    `G` is an instance of :class:`networkx.Graph`.

    `func` is a function on two inputs, each of which is a node in the
    graph. The function can return anything, but it should return a
    value representing a prediction of the likelihood of a "link"
    joining the two nodes.

    `ebunch` is an iterable of pairs of nodes. If not specified, all
    non-edges in the graph `G` will be used.

    """
    if ebunch is None:
        ebunch = nx.non_edges(G)
    return ((u, v, func(u, v)) for u, v in ebunch)

In [8]:
def jaccard_coefficient(G, ebunch=None):
    """Compute the Jaccard coefficient of all node pairs in ebunch.

    Jaccard coefficient of nodes `u` and `v` is defined as

    .. math::

        \frac{|\Gamma(u) \cap \Gamma(v)|}{|\Gamma(u) \cup \Gamma(v)|}

    where $\Gamma(u)$ denotes the set of neighbors of $u$.

    Parameters
    ----------
    G : graph
        A NetworkX undirected graph.

    ebunch : iterable of node pairs, optional (default = None)
        Jaccard coefficient will be computed for each pair of nodes
        given in the iterable. The pairs must be given as 2-tuples
        (u, v) where u and v are nodes in the graph. If ebunch is None
        then all non-existent edges in the graph will be used.
        Default value: None.

    Returns
    -------
    piter : iterator
        An iterator of 3-tuples in the form (u, v, p) where (u, v) is a
        pair of nodes and p is their Jaccard coefficient.

    Examples
    --------
    >>> import networkx as nx
    >>> G = nx.complete_graph(5)
    >>> preds = nx.jaccard_coefficient(G, [(0, 1), (2, 3)])
    >>> for u, v, p in preds:
    ...     '(%d, %d) -> %.8f' % (u, v, p)
    ...
    '(0, 1) -> 0.60000000'
    '(2, 3) -> 0.60000000'

    References
    ----------
    .. [1] D. Liben-Nowell, J. Kleinberg.
           The Link Prediction Problem for Social Networks (2004).
           http://www.cs.cornell.edu/home/kleinber/link-pred.pdf
    """
    def predict(u, v):
        union_size = len(set(G[u]) | set(G[v]))
        if union_size == 0:
            return 0
        return len(list(nx.common_neighbors(G, u, v))) / union_size
    return _apply_prediction(G, predict, ebunch)

In [9]:
def sorensen_index(G, ebunch=None):
    """Compute the Sorensen Index of all node pairs in ebunch.

    Sorensen Index of nodes `u` and `v` is defined as

    .. math::

        \frac{|\Gamma(u) \cap \Gamma(v)|}{|\Gamma(u)| + |\Gamma(v)|}

    where $\Gamma(u)$ denotes the set of neighbors of $u$.

    Parameters
    ----------
    G : graph
        A NetworkX undirected graph.

    ebunch : iterable of node pairs, optional (default = None)
        Jaccard coefficient will be computed for each pair of nodes
        given in the iterable. The pairs must be given as 2-tuples
        (u, v) where u and v are nodes in the graph. If ebunch is None
        then all non-existent edges in the graph will be used.
        Default value: None.

    Returns
    -------
    piter : iterator
        An iterator of 3-tuples in the form (u, v, p) where (u, v) is a
        pair of nodes and p is their Jaccard coefficient.
    """
    def predict(u, v):
        union_size = len(set(G[u])) + len(set(G[v]))
        if union_size == 0:
            return 0
        return len(list(nx.common_neighbors(G, u, v))) / union_size
    return _apply_prediction(G, predict, ebunch)

In [10]:
def salton_cosine_similarity(G, ebunch=None):
    """Compute the Salton Cosine Similarity of all node pairs in ebunch.

    Salton Cosine Similarity of nodes `u` and `v` is defined as

    .. math::

        \frac{|\Gamma(u) \cap \Gamma(v)|}{\sqrt{|\Gamma(u)| \times |\Gamma(v)|}}

    where $\Gamma(u)$ denotes the set of neighbors of $u$.

    Parameters
    ----------
    G : graph
        A NetworkX undirected graph.

    ebunch : iterable of node pairs, optional (default = None)
        Jaccard coefficient will be computed for each pair of nodes
        given in the iterable. The pairs must be given as 2-tuples
        (u, v) where u and v are nodes in the graph. If ebunch is None
        then all non-existent edges in the graph will be used.
        Default value: None.

    Returns
    -------
    piter : iterator
        An iterator of 3-tuples in the form (u, v, p) where (u, v) is a
        pair of nodes and p is their Jaccard coefficient.
    """
    def predict(u, v):
        import math
        union_size = math.sqrt(len(set(G[u])) * len(set(G[v])))
        if union_size == 0:
            return 0
        return len(list(nx.common_neighbors(G, u, v))) / union_size
    return _apply_prediction(G, predict, ebunch)

In [11]:
def preferential_attachment(G, ebunch=None):
    """Compute the Preferential Attachment of all node pairs in ebunch.

    Preferential Attachment of nodes `u` and `v` is defined as

    .. math::

        \frac{|\Gamma(u) \cap \Gamma(v)|}{\sqrt{|\Gamma(u)| \times |\Gamma(v)|}}

    where $\Gamma(u)$ denotes the set of neighbors of $u$.

    Parameters
    ----------
    G : graph
        A NetworkX undirected graph.

    ebunch : iterable of node pairs, optional (default = None)
        Jaccard coefficient will be computed for each pair of nodes
        given in the iterable. The pairs must be given as 2-tuples
        (u, v) where u and v are nodes in the graph. If ebunch is None
        then all non-existent edges in the graph will be used.
        Default value: None.

    Returns
    -------
    piter : iterator
        An iterator of 3-tuples in the form (u, v, p) where (u, v) is a
        pair of nodes and p is their Jaccard coefficient.
    """
    def predict(u, v):
        return len(set(G[u])) * len(set(G[v]))
    return _apply_prediction(G, predict, ebunch)

In [12]:
def rank(pred, k):
    """
    Parameters
    ----------
    pred : An iterator of 3-tuples in the form (u, v, p) where (u, v) is a
        pair of nodes and p is their Jaccard coefficient.

    k : keep top k element based p

    Returns
    -------
    top k list
    """
    l = [(u, v, p) for u, v, p in pred]
    
    sorted_l = sorted(l, key=lambda x: x[2], reverse=True)
    return sorted_l[0:k]

In [13]:
splitFile('email-Eu-core-temporal', 0.8)

In [14]:
email_train_g = loadGraph('email-Eu-core-temporal_train')

In [32]:
pred = salton_cosine_similarity(email_train_g)
res = rank(pred, 100)
# print(res)

In [16]:
len(res)

100

In [36]:
# evaluating prediction quality

def prediction_eval(G, true_links, pred_links):
    """
    G: network before prediction
    """
    true_links = set(true_links)
    pred_links = set(pred_links)
    TP = 0; TN = 0; FP = 0; FN = 0

    for u, v in nx.non_edges(G):
        if ((u,v) in true_links or (v,u) in true_links) and ((u,v) in pred_links or (v,u) in pred_links):
            TP += 1
        elif (not ((u,v) in true_links or (v,u) in true_links)) and ((u,v) in pred_links or (v,u) in pred_links):
            FP += 1
        elif ((u,v) in true_links or (v,u) in true_links) and not (((u,v) in pred_links or (v,u) in pred_links)):
            FN += 1
        else:
            TN += 1
    
#     print (TP, FP, FN, TN)
    precision = TP / (TP + FP)
    return precision

In [28]:
# load true links
def load_true_links(filename):
    # load true links
    true_links = []
    with open(filename, "r") as fp:
        line = fp.readline()

        # skip comments
        while line[0] == '#':
            line = fp.readline()

        while line:
            edge = line.split()
            true_links.append((edge[0], edge[1]))
            line = fp.readline()
    
    return true_links

In [39]:
# TP, FP, FN, TN

pred = preferential_attachment(email_train_g)
res = rank(pred, 2000)

pred_pairs = [(u,v) for u,v,p in res]
true_links = load_true_links("dataset/email-Eu-core-temporal_test.txt")
precision = prediction_eval(email_train_g, true_links, pred_pairs)
print(precision)

0.067


In [17]:
preds = nx.jaccard_coefficient(email_train_g)
count = 0
l = []
for u, v, p in preds:
    if p > 0:
        l.append((u, v, p))
#     print('(%s, %s) -> %.8f' % (u, v, p))

KeyboardInterrupt: 

In [57]:
G = nx.complete_graph(5)
preds = nx.jaccard_coefficient(G)
for u, v, p in preds:
    print('(%d, %d) -> %.8f' % (u, v, p))